<a href="https://colab.research.google.com/github/srilalithaveerubhotla/Trash-Management-System/blob/master/trashmanagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**TRASH MANAGEMENT SYSTEM**

[<img src="https://github.com/srilalithaveerubhotla/Trash-Management-System/blob/master/TrashImages.jpg?raw=true" width="1000" height="300" align="center">]()

[<img src="https://github.com/shubham0204/Privacy_Policy_Texts/blob/master/notebook_button_two.png?raw=true" width="150" height="50" align="center">](https://github.com/garythung/trashnet)



[<img src="https://github.com/srilalithaveerubhotla/Trash-Management-System/blob/master/tensorboard-logo-social.png?raw=true" width="300" height="200" align="right">](https://tensorboard.dev/experiment/9OqkrQuOT4uvh0JNSmGf8g/#scalars&run=20200516-232647%2Ftrain)
## **This is a final project code for CMPE 258 Final Project Course work**

###Guided By : Vijay Eranti

###Team Memebers : 

###1.   Srilalitha Veerubhotla
###2.   Shreyus Puthiyarapurail
###3.   Atul Shah
###4.   Shailesha Prasad Maganahalli

Idea of this project Trash Classification which we used the dataset which has six classes: glass, paper, cardboard, plastic, metal, and trash. Currently, the dataset consists of 2527 images:

501 glass

*  501 glass
*  594 paper
*  403 cardboard
*  482 plastic
*  410 metal
*  137 trash










which this pictures are already been taken by the auther mentioned in the above link.
We wrote distinct variety of Deep Learning Models to address the basic image classification technique.

---


This notebook Follows as below..

1. Data Importing
2. Data Pre-Processing
3. Data Visualisations with TensorBoard Dev
4. CNN ARCHITECTURES with Hyper parameter Tuning 
5. Metrics and Loss Functions 















##DATA IMPORTING 

---versioning----

Keras Version:  >3.0

TensorFlow version:  2.2.0

TensorBoard


In [0]:
import cv2
import numpy
import glob
import pylab as plt
import os
import keras.backend as K


# importing libraries
from matplotlib import pyplot
from keras import datasets
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# importing Image class from PIL package  
from PIL import Image  


from keras.callbacks import TensorBoard

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version
from PIL import Image


import numpy as np
import cv2
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import matplotlib.pyplot as plt

#from kerastuner.tuners import RandomSearch
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from tensorboard.plugins.hparams import api as hp
from keras.utils import print_summary, to_categorical

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."
from datetime import datetime
%load_ext tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir {logs_base_dir}

In [0]:
# Mounting Drive and accessing the datafiles required

from google.colab import drive
drive.mount('/content/drive')


Load Garbage Classification zip file to google drive 

In [0]:
# Unzipping the file 

!unzip /content/drive/My\ Drive/garbage-classification.zip

Annotated Images are stored in different files as per thier labels to access the files and their annotations from folder names below lines of code is used 

In [0]:
# Importing and deriving the training images and labelling them
trash_images = []
labels = [] 
for fruit_dir_path in glob.glob("/content/garbage classification/Garbage classification/*/"):
    fruit_label = fruit_dir_path.split("/")[4]
    
    # Path for reading the files
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.jpg")):

      # Using cv2 readig the images from path
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)

        # Resizing the images
        image = cv2.resize(image, (70, 70))

        # Colour Transformations 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        trash_images.append(image)
        labels.append(fruit_label)
        # Appending and deriving the final images 

trash_images = np.array(trash_images)
labels = np.array(labels)

In [0]:
# Converting the Key-value dictonaries of the annotations into labels 

label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [0]:
training_label_id = np.array([label_to_id_dict[i] for i in labels])

In [0]:
# Checking the shape of the data 

print("Train_data shape......:",trash_images.shape)
print("Labels shape..........:",training_label_id.shape)

In [0]:
# Splitting the data into Train and Test

(trainX, testX, trainY, testY) = train_test_split(trash_images, training_label_id, test_size=0.25)

In [0]:
# Tensor Board Integration by giving the approriate log location with the time stamp 

now = datetime.now()
logdir = "logs/trash_images/" + now.strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
with file_writer.as_default():
  tf.summary.image("Training data", np.reshape(trainX[0:10],(-2,70,70,3)), step=0)
file_writer.set_as_default()

In [0]:
# Execution of TensorBoard at every step

%tensorboard --logdir logs/trash_images/

In [0]:
# Path for the image folders

dir_path = '/content/garbage classification/Garbage classification'

In [0]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [0]:
# Checking the Length of the images

len(img_list)

In [0]:
# VISUALISING THE IMAGE 
  
# creating a object  
Image.open('/content/garbage classification/Garbage classification/plastic/plastic136.jpg')  


## IMAGE ARGUMENTATION
Arranging the image size, scale, width, height, and rescale, flipping 

In [0]:
## ---------Using Image Data Generator below code allows you to do the argumentation on the -----------------
##------------ mentioned parameters and sets your image positions---------------------

#_------------------- Generating for Train Image------------------
train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

# ------------------Generating for test Image ------------------------

test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

train_generator=train.flow_from_directory(dir_path,
                                          target_size=(300,300),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

test_generator=test.flow_from_directory(dir_path,
                                        target_size=(300,300),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='validation')

#---------------------------Generating the validation dataset -------------------

validation_generator = test.flow_from_directory(
    dir_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

labels = (train_generator.class_indices)
print(labels)

labels = dict((v,k) for k,v in labels.items())
print(labels)


 LABELS: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

Train Images : 2276

Test Images :  251

In [0]:
# From the generator creating the labels and actual images 

for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

In [0]:
print (train_generator.class_indices)

Labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(Labels)

## Simple CNN Model 

In [0]:


# SETTING THE SEEDS FOR RANDOM 
tf.random.set_seed(42)
np.random.seed(42)

model=tf.keras.Sequential()

#------------------------------CONVOLUTIONAL BLOCKS------------------------------

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

model.add(tf.keras.layers.Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

#--------------------------CLASSIFICATION LAYERS--------------------------------
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
#-----------------------------OUTPUT DENSE LAYER-------------------------------
model.add(tf.keras.layers.Dense(6,activation='softmax'))

# FILEPATH FOR SAVING THE MODEL LOGS
filepath="/content/trained_model.h5"
logdir="logs/trash_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")

#TENSORBOARD INTEGRATION
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# MODEL CHECK POINTS
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("trash_simple_model.h5", save_best_only=True)
callbacks_list = [model_checkpoint,tensorboard_callback]

In [0]:
#---------------MODEL SUMMARY-------------
model.summary()

In [0]:
# ---------------PLOTTING THE MODEL AS PNG FILE ------------
keras.utils.plot_model(model, "simple_cnn_trash.png", show_shapes=True)

In [0]:
# -------------------Compiling the model with categorical cross entropy as loss function and 
#. ADAM as optimiser with acuuracy and Mean Square Error and metrics---------------------------------------

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc','mse']) 

In [0]:
#---------------- MODEL FITTING WITH EPOCHS, WORKERS, BY INITIATING THE CALL BACKS AND VALIDATION ON DATA---------------------

history = model.fit_generator(train_generator,
                              epochs=10,
                              steps_per_epoch=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers = 4,
                              callbacks=callbacks_list) 

In [0]:
#------------PRINTING THE LEARNING RATE USED----------------------
print(K.eval(model.optimizer.lr))

In [0]:
#-----------------------UPLOADING THE LOGS TO TENSORBOARD DEV-----------------------
!tensorboard dev upload --logdir logs/trash_images/ \
  --name "Trash Management" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb"

## Simple CNN Fine Tuned Model

In [0]:
model=tf.keras.Sequential()

#------------------------------THREE CONVOLUTIONAL BLOCKS------------------------------
model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 
model.add(tf.keras.layers.Dropout(0.5)) # No accuracy

model.add(tf.keras.layers.Conv2D(64,(3,3), padding='same',activation='selu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',activation='selu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

#-------------------------- FOUR CLASSIFICATION LAYERS WITH DROPOUT--------------------------------

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(126,activation='selu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(64,activation='selu'))

model.add(tf.keras.layers.Dropout(0.5))


model.add(tf.keras.layers.Dense(32,activation='selu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(16,activation='selu'))
model.add(tf.keras.layers.Dropout(0.5))

#-----------------------------OUTPUT DENSE LAYER-------------------------------

model.add(tf.keras.layers.Dense(6,activation='softmax'))


# FILEPATH FOR SAVING THE MODEL LOGS
filepath="/content/trained_model.h5"
logdir="logs/trash_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")

#TENSORBOARD INTEGRATION
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# MODEL CHECK POINTS
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("trash_simple_model1.h5", save_best_only=True)
callbacks_list = [model_checkpoint,tensorboard_callback]

In [0]:
#---------------MODEL SUMMARY-------------
model.summary()

In [0]:
# ---------------PLOTTING THE MODEL AS PNG FILE ------------

keras.utils.plot_model(model, "simple_cnn1_trash.png", show_shapes=True)

In [0]:
#---------------- MODEL COMPILING RMS PROP OPTIMISER WITH LEARNING RATE AS 0.0003 AND LOSS FUNCTION AS KULLBACK LEIBER DIVERGENCE 

opt = tf.keras.optimizers.RMSprop(learning_rate=0.003)
model.compile(loss='kullback_leibler_divergence', optimizer=opt, metrics=['acc'])

In [0]:
#---------------- MODEL FITTING WITH EPOCHS, WORKERS, BY INITIATING THE CALL BACKS AND VALIDATION ON DATA---------------------


history = model.fit_generator(train_generator,
                              epochs=10,
                              steps_per_epoch=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers = 10,
                              callbacks=callbacks_list) 

In [0]:
print(K.eval(model.optimizer.lr))

In [0]:
!mkdir -p classifier1

In [0]:

#-----------------SAVING THE MODEL-----------------
model_save_name = 'classifier1.h5'
model.save('/content/trained_model.h5')

In [0]:
#----------------SAVING THE MODEL AS JSON FILE---------------
model_json = model.to_json()
with open("/content/model.json", "w") as json_file:
    json_file.write(model_json)

### VGG16 TRANSFER LEARNING Using Pretrained TOP NOTCH MODEL

In [0]:
#---------------------------NECESSARY IMPORTS FOR TRANSFER LEARNING PRETRAINED MODEL FOR VGG16---------------------------

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, losses
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import pickle

In [0]:
#-----------------------INSTANTIATING VGG16 TRANSFERLEARNED MODEL ----------------------
model = VGG16()
keras.utils.plot_model(model, "simple_vgg16_tl_trash.png", show_shapes=True)

In [0]:

print(model.summary())

In [0]:
from keras.preprocessing.image import load_img
# load an image from file
image = load_img('/content/garbage classification/Garbage classification/paper/paper357.jpg', target_size=(224, 224))


In [0]:
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)

In [0]:
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

In [0]:
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image = preprocess_input(image)

In [0]:
yhat = model.predict(image)

In [0]:
from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

In [0]:
!unzip /content/drive/My\ Drive/keras-pretrained-models.zip

In [0]:
#------------------USING THE SAVED MODEL VGG16 WEIGHTS IMPORTING AS A BASE MODEL FOR TRANSFER LEARNING----------------

#------------------------------------DEFINING MORE LAYERS ON TOP OF VGG16 FOR TRANSFER LEARNING----------------------------


base_model = VGG16(weights=None, include_top=False, input_shape=(70, 70, 3))
base_model.load_weights('/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_model.trainable = False

In [0]:
#--------------------------generating necessary training and testing data---------------------
gen = {
    "train": ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1. / 255,
        validation_split=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=30,
    ).flow_from_directory(
        directory=dir_path,
        target_size=(300, 300),
        subset='training',
    ),

    "valid": ImageDataGenerator(
        rescale=1 / 255,
        validation_split=0.1,
    ).flow_from_directory(
        directory=dir_path,
        target_size=(300, 300),
        subset='validation',
    ),
}

In [0]:
#------------------------------SEQUENTIAL TENSORFLOW KERAS MODEL AND ADDING BASE MODEL TO IT--------------------
#-----------------------ADDING GROUPAVERAGEPOOLING--------------------
#-----------------------ADDING DROPOUT & DENSE LAYERS-----------------
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.15),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')
])

opt = tf.keras.optimizers.Nadam(learning_rate=0.002)

#-------------MODEL COMPILING---------------------
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy','mse'])

#-----------------------EPOCHS AND BATCH SIZE----------------
batch_size = 200
epochs = 10
train_generator = gen["train"]
valid_generator = gen["valid"]

#-------------------------ADDING STEP EPOCHS AND VALIDATION SETPS FOR CALL BACKS--------------------------
steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size


#--------------------------MODEL SAVING, CHECK POINTING AND TENSORBOARD INTGRATION--------------------
filepath = "/content/vggpretrained.h5"

logdir="logs/trash_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("trash_simple_model1.h5", save_best_only=True)
callbacks_list = [model_checkpoint,tensorboard_callback]

#--------------------------------MODEL FITTING----------------------------------------
history = model.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator, validation_steps=validation_steps,
                              callbacks=callbacks_list)
with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [0]:
!tensorboard dev upload --logdir logs/trash_images/ \
  --name "Trash Management" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb"

## VGG 16 From Scratch code

In [0]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [0]:
#DEFINING MODEL SHAPE
input_shape=(300,300,3)

#---------------------STARTING THE SEQUENTIAL MODEL--------------------
model =tf.keras.Sequential(name='VGG-16')

# --------------------------------------CONVOLUTIONAL LAYERS WITH BATCH NORMALIZATION, DROPOUT,---------------------------------------------------

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same',input_shape=input_shape))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
#-------------------MAXPOOLING LAYER------------------------
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


model.add(tf.keras.layers.Conv2D(512, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(512, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(512, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


model.add(tf.keras.layers.Conv2D(512, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(512, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Conv2D(512, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.5))

#--------------------------------------CLASSIFICATION LAYERS--------------------------------

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

#-----------------------------OUTPUT DENSE LAYER-------------------------------
model.add(tf.keras.layers.Dense(6))

#------------------------------LOGISTIC REGRESSION LAYER----------------------
model.add(tf.keras.layers.Activation('softmax'))



In [0]:
keras.utils.plot_model(model, "simple_vgg16_scratch_trash.png", show_shapes=True)

In [0]:
# -------------------Compiling the model with BINARY cross entropy as loss function and 
#. ADAM as optimiser with acuracy metrics---------------------------------------

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
# train_generator = gen["train"]
# valid_generator = gen["valid"]
# batch_size = 200
# epochs = 10

# steps_per_epoch = train_generator.n // batch_size
# validation_steps = valid_generator.n // batch_size
logdir="logs/trash_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("trash_simple_model1.h5", save_best_only=True)
callbacks_list = [model_checkpoint,tensorboard_callback]

In [0]:
#---------------- MODEL FITTING WITH EPOCHS, WORKERS, BY INITIATING THE CALL BACKS AND VALIDATION ON DATA---------------------

history = model.fit_generator(train_generator,
                              epochs=10,
                              steps_per_epoch=2276//32,
                              validation_data=validation_generator,
                              validation_steps=251//32,
                              workers = 4,
                              callbacks=callbacks_list) 

## INCEPTION V3 TOP Notch Transfer Learned

In [0]:

#---------------------------NECESSARY IMPORTS FOR TRANSFER LEARNING PRETRAINED MODEL FOR INCEPTIONV3---------------------------

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import InceptionV3
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras import optimizers, losses
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import pickle

In [0]:
#--------------------------INITIATING PRE-TRAINED INCEPTION MODEL WITH WEIGHTS---------------------------

base_model = InceptionV3(weights=None, include_top=False, input_shape=(300, 300, 3))
base_model.load_weights('/content/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_model.trainable = False


In [0]:
#----------------------------------ADDING DENSE, DROPOUT AND POOLONG LAYERS TO THE PRETRAINED MODELS FOR TRANSFER LEARNING--------------------

#------------------------------------DEFINING MORE LAYERS ON TOP OF INCEPTIONV3 FOR TRANSFER LEARNING----------------------------

model = Sequential([    
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.15),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')
])
#----------------------RMSPROP OPTIMIZER WITH LEARNING RATE----------------------------
opt = optimizers.rmsprop(lr=0.0002)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

#----------------------BATCH SIZING AND EPOCH-----------------------
batch_size = 300
epochs = 10
train_generator = gen["train"]
valid_generator = gen["valid"]

steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

#--------------------------------------- FILEPATH FOR SAVING THE MODEL LOGS------------------------

filepath = "/content/model_inception_pre.h5"


# --------------------------------MODEL CHECK POINTS------------------------------------------------
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

#-----------------------------------FITIING MODEL----------------------------------------
history = model.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator, validation_steps=validation_steps,
                              callbacks=callbacks_list)
with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## RESNET 50 top notch Transfer Learning

In [0]:
#---------------------------NECESSARY IMPORTS FOR TRANSFER LEARNING PRETRAINED MODEL FOR RESNET50---------------------------


from keras.applications.resnet50 import ResNet50
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras import optimizers, losses
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import pickle


In [0]:
#--------------------------INITIATING PRE-TRAINED INCEPTION MODEL WITH WEIGHTS---------------------------

#------------------------------------DEFINING MORE LAYERS ON TOP OF RESNET50 FOR TRANSFER LEARNING----------------------------


base_model = ResNet50(weights=None, include_top=False, input_shape=(300, 300, 3))
base_model.load_weights('/content/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
base_model.trainable = False

In [0]:
#----------------------------------ADDING DENSE, DROPOUT AND POOLONG LAYERS TO THE PRETRAINED MODELS FOR TRANSFER LEARNING--------------------


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.15),
    Dense(1024, activation='selu'),
    Dense(6, activation='softmax')
])

#----------------------ADAGRAD OPTIMIZER WITH LEARNING RATE----------------------------

opt = optimizers.adagrad(lr=0.0003)

#--------------------------MODEL COMPILING-----------------
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

#----------------------BATCH SIZING AND EPOCH-----------------------
batch_size = 350
epochs = 10
train_generator = gen["train"]
valid_generator = gen["valid"]

steps_per_epoch = train_generator.n // batch_size
validation_steps = valid_generator.n // batch_size

# --------------------------------MODEL CHECK POINTS------------------------------------------------
filepath = "/content/model_resnetpre.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

#-------------------------------TRAINING MODEL--------------------------------------
history = model.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                              validation_data=valid_generator, validation_steps=validation_steps,
                              callbacks=callbacks_list)
with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## RESNET 50 SCRATCH CODE

In [0]:

#---------------------------NECESSARY IMPORTS FOR TRANSFER LEARNING PRETRAINED MODEL FOR RESNET50---------------------------


import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [0]:
def identity_block(X, f, filters, stage, block):
    
    # ---------------------Defining name basis----------------------------
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # ------------------------Retrieve Filters---------------------------------
    F1, F2, F3 = filters
    
    # ---------------------------SAVE THE INPUT FILES------------------------
    X_shortcut = X
    
    #---------------------- First component of main path---------------------
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # -----------------------Second component of main path-------------------------
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # ---------------------Third component of main path ------------------------------
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # ---------------------------Final step: Add shortcut value to main path, and pass it through a RELU activation---------------------
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s=2):

    # ---------------------------Defining name basis--------------------------
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # ---------------------------Retrieve Filters--------------------------
    F1, F2, F3 = filters

    # ----------------------------Save the input value----------------------
    X_shortcut = X

    ##### MAIN PATH #####
    # ---------------------------FIRST COMPONENT---------------------------------
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # --------------------------SEOND COMPONENT------------------------------------
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # --------------------------THIRD COMPONENT------------------------------------
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    #####--------------------------- SHORTCUT PATH----------------------------- #### 
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # -------------------------Final step: Add shortcut value to main path, and pass it through a RELU activation----------------------------------
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [0]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    
    # --------------------Define the input as a tensor with shape input_shape----------------------
    X_input = Input(input_shape)

    
    # --------------------------Zero-Padding--------------------------------------------
    X = ZeroPadding2D((3, 3))(X_input)
    
    # -------------------------------STAGE 1-----------------------------------------
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # --------------------------------STAGE 2-----------------------------------------
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # ----------------------------------STAGE 3-----------------------------------------
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # ----------------------------------STAGE 4------------------------------------------
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # ---------------------------------STAGE 5----------------------------------------
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    #------------------------- AVGPOOL------------------------
    X = AveragePooling2D(pool_size=(2,2), padding='same')(X)

    #------------------- OUTPUT DENSE LAYER---------------------
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # -----------------------MODEL CREATING--------------
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

### DATA NORMALISATION AND CONVERTING LABELS INTO CATEGORIES 

In [0]:
# -----------------------------------Normalize image vectors------------------------------
X_train = trainX/255.
X_test = testX/255.

#-------------------------------Convert training and test labels to one hot matrices---------------------
Y_train = to_categorical(trainY, 6)
Y_test = to_categorical(testY, 6)

In [0]:
#--------------------------------- PRINTING THE SHAPE OF THE TRAIN AND TEST DATA----------------------------

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [0]:
#------------------------------------------------------------COMPILING THE SCRATCH CODE FUNCTION--------------------
model = ResNet50(input_shape = (70, 70, 3), classes = 6)
keras.utils.plot_model(model, "simple_RESNET_trash.png", show_shapes=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
#--------------------------------------------training model------------------------------------
model.fit(X_train,Y_train, epochs = 5)

## MobileNet Transfer Learned Top Notch

In [0]:
#------------------------------INSTANTIATING THE MOBILENETV2 PRE-TRAINED MODEL ------------------------------------
IMG_SHAPE = (224,224,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
base_model.trainable = False


In [0]:
#------------------------------------DEFINING MORE LAYERS ON TOP OF MOBILENETV2 FOR TRANSFER LEARNING----------------------------
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [0]:
#-------------------------------------------------------COMPILING THE CODE----------------------------------
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), #Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
#----------------------------------------BATCH SIZING, EPOCHS & CHECKING POINTING-----------------------------
batch_size = 32
epochs = 50
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

#----------------------------TRAINING THE MODEL----------------------------------------------

history = model.fit_generator(train_generator, 
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs, 
                              workers=4,
                              validation_data=validation_generator, 
                              validation_steps=validation_steps)


In [0]:
#----------------MODEL SAVING--------------------------
model_save_name = 'mobilenet-transfer.h5'
model.save('/content/mobilenet-transfer.h5')